## Calculate assay enrichment statistics for all chemical clusters
Quantify if a chemcial cluster has a signficiant effect beyond the background hitrate on an assay or not.

In [ ]:
import sqlite3 
import pandas as pd
import glob
import os

from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
import numpy as np

In [ ]:
conn = sqlite3.connect('../pubchem_gcm.db')

### Calculate activitiy counts over assays
* Calculate the numbers of actives and inactives in both assay directions for all assays
* Calculate the same numbers for each chemcial cluster to compare if clusters contain a larger than expected number of hits for an assay / assay direction
* Filter out assays which have less than 30% of the cluster cpds measured relative to the max amount of cpds measured by an assay to keep profiles comparable

In [ ]:
df_assay_gcm_stat = pd.read_sql("""

    WITH cpds_data AS (
        SELECT a.aid, a.cid, a.rscore, g.gcm_cluster, g.cluster_size
        FROM  assays a
        JOIN gcm_clusters g ON (a.cid=g.cid)
        --WHERE g.gcm_cluster IN (7868,7869,7870) 
    ), 
    -- calculate background counts of actives and inactives per aid
    bg_num_tested AS (
        SELECT cpd.aid, count(*) AS bg_num_tested
        FROM   cpds_data cpd
        GROUP BY cpd.aid
    ),
    bg_num_up AS (
        SELECT cpd.aid, count(*) AS bg_num_actives
        FROM   cpds_data cpd
        WHERE  cpd.rscore > 3
        GROUP BY cpd.aid
    ),
    bg_num_down AS (
        SELECT cpd.aid, count(*) AS bg_num_actives
        FROM   cpds_data cpd
        WHERE  cpd.rscore < -3
        GROUP BY cpd.aid
    ),
    bg_stat_up AS (
        SELECT t.*, COALESCE(u.bg_num_actives,0) AS bg_num_actives, 
                    COALESCE(t.bg_num_tested - (u.bg_num_actives),0) AS bg_num_inactives
        FROM bg_num_tested t
        JOIN bg_num_up u ON (t.aid=u.aid)
    ),
    bg_stat_down AS (
        SELECT t.*, COALESCE(d.bg_num_actives,0) AS bg_num_actives, 
                    COALESCE(t.bg_num_tested - (d.bg_num_actives),0) AS bg_num_inactives
        FROM bg_num_tested t
        JOIN bg_num_down d ON (t.aid=d.aid)
    ),
    -- calculate actives and inactives counts per gcm cluster
    gcm_num_tested AS (
        SELECT cpd.aid, cpd.gcm_cluster, cpd.cluster_size, count(*) AS num_tested
        FROM   cpds_data cpd
        GROUP BY cpd.aid, cpd.gcm_cluster, cpd.cluster_size
    ),
    gcm_num_actives_up AS (
        SELECT cpd.aid, cpd.gcm_cluster, cpd.cluster_size, count(*) AS num_actives
        FROM   cpds_data cpd
        WHERE  cpd.RSCORE > 3
        GROUP BY cpd.aid, cpd.gcm_cluster, cpd.cluster_size
    ),
    gcm_num_actives_down AS (
        SELECT cpd.aid, cpd.gcm_cluster, cpd.cluster_size, count(*) AS num_actives
        FROM   cpds_data cpd
        WHERE  cpd.RSCORE < -3
        GROUP BY cpd.aid, cpd.gcm_cluster, cpd.cluster_size
    ),
    gcm_max_num_tested AS (
        SELECT t.gcm_cluster, MAX(t.num_tested) AS max_cpds_tested
        FROM  gcm_num_tested t
        GROUP BY t.gcm_cluster
    )
    -- concatenate up and down data
    SELECT t.aid, 'up' AS activity_direction, 1 AS act_dir,
                t.gcm_cluster, t.cluster_size, m.max_cpds_tested, 
                t.num_tested, 
                (CASE WHEN (t.num_tested / m.max_cpds_tested) > 0.3 THEN 'yes'
                    ELSE 'no' END) AS assay_qualified_for_profile,
                COALESCE(a.num_actives,0) AS num_actives,  
                t.num_tested - COALESCE(a.num_actives,0) AS num_inactives,
                COALESCE(1.0*a.num_actives / t.num_tested ,0.0) AS fract_actives,
                bg.bg_num_actives, bg.bg_num_inactives
    FROM gcm_num_tested t
    LEFT JOIN gcm_num_actives_up a ON (t.aid=a.aid AND t.gcm_cluster=a.gcm_cluster AND t.cluster_size=a.cluster_size)
    JOIN bg_stat_up bg USING (aid)
    JOIN gcm_max_num_tested m USING (gcm_cluster)
    
    UNION
    
    SELECT t.aid, 'down' AS activity_direction, -1 AS act_dir,
                t.gcm_cluster, t.cluster_size, m.max_cpds_tested, 
                t.num_tested, 
                (CASE WHEN (t.num_tested / m.max_cpds_tested) > 0.3 THEN 'yes'
                    ELSE 'no' END) AS assay_qualified_for_profile,
                COALESCE(a.num_actives,0) AS num_actives,  
                t.num_tested - COALESCE(a.num_actives,0) AS num_inactives,
                COALESCE(1.0*a.num_actives / t.num_tested ,0.0) AS fract_actives,
                bg.bg_num_actives, bg.bg_num_inactives
    FROM gcm_num_tested t
    LEFT JOIN gcm_num_actives_down a ON (t.aid=a.aid AND t.gcm_cluster=a.gcm_cluster AND t.cluster_size=a.cluster_size)
    JOIN bg_stat_down bg USING (aid)
    JOIN gcm_max_num_tested m USING (gcm_cluster)
    
;""", conn)

In [ ]:
df_assay_gcm_stat.to_csv('assay_gcm_cnts.csv', index=False)

In [ ]:
df_assay_gcm_stat = pd.read_csv('assay_gcm_cnts.csv', low_memory=False)

### Calculate p-values 

In [ ]:
df_assay_gcm_stat['p_val'] = df_assay_gcm_stat.apply(
    lambda r: fisher_exact([[r.num_actives, r.num_inactives], [r.bg_num_actives, r.bg_num_inactives]], alternative='greater')[1],
    axis=1)

In [ ]:
df_assay_gcm_stat['adj_p_val'] = multipletests(df_assay_gcm_stat['p_val'], method='fdr_bh', is_sorted=False, returnsorted=False)[1]

In [ ]:
df_assay_gcm_stat["neg_log_adj_p_val"] = df_assay_gcm_stat["adj_p_val"].apply(lambda x: np.min([350, -np.log10(x)]))

### Upload stats to db

In [ ]:
conn.execute("""DROP TABLE IF EXISTS gcm_cluster_assay_stat""")

# create table with keys before and add via pandas
conn.execute('''
CREATE TABLE gcm_cluster_assay_stat(
        aid INT,
        activity_direction TEXT,
        act_dir INT,
        gcm_cluster INT,
        cluster_size INT,
        max_cpds_tested INT,
        num_tested INT,
        assay_qualified_for_profile TEXT,
        num_actives INT,
        num_inactives INT,
        fract_actives REAL,
        bg_num_actives INT,
        bg_num_inactives INT,
        p_val REAL,
        adj_p_val REAL,
        neg_log_adj_p_val REAL,
        PRIMARY KEY(gcm_cluster, aid, act_dir)
         );
         ''')

In [ ]:
df_assay_gcm_stat.to_sql('gcm_cluster_assay_stat', conn, if_exists='append', index=False) 

In [ ]:
conn.execute('''CREATE INDEX assay_stat_aid_act_dir_index ON gcm_cluster_assay_stat (aid, act_dir);''')

In [ ]:
conn.close()